In [1]:
import pandas as pd
import numpy as np
from ortools.sat.python import cp_model
from datetime import datetime, timedelta

In [2]:
import sys
sys.path.append('/home/cayena/Projeto_thefamily/TeacherScheduler/')

In [3]:
from streamlit_app.utils import transform_classes_dateframe,transform_teacher_dataframe,transform_alocation_dataframe,base_selection,expand_rows,replicate_row,clean_data
from streamlit_app.teacher_alocation import TeacherScheduler

In [4]:
aulas_raw = pd.read_excel('/home/cayena/Downloads/PADRÃO DA ROTA 2025.xlsx')
professores_raw = pd.read_excel('/home/cayena/Downloads/PADRÃO PROFESSORES 2025.xlsx')


In [5]:
df_result =transform_classes_dateframe(aulas_raw)
df_professor = transform_teacher_dataframe(professores_raw)

In [6]:
Ts = TeacherScheduler(df_result, professores_raw)
base_alocada = Ts.schedule_teachers()
base_alocada

42172
Não foi possível encontrar uma solução ótima.


,professores_alocados,nome grupo


In [7]:
df_result_filtrado = df_result[((df_result['teacher']!='-')&(df_result['teacher'].notnull()))]

In [15]:
#horario dia semana iguais outra aulas
for i in df_result_filtrado['teacher'].unique():
    teste = df_result_filtrado[df_result_filtrado['teacher']==i]
    for a in teste['dias da semana'].unique():
        for h in teste['horario'].unique():
            shape = teste[(teste['dias da semana']==a)&(teste['horario']==h)]['horario'].nunique()
            if shape > 1:
                print(i,a,shape)


In [ ]:
#professor nao pode dar aula no horario
for i in df_result_filtrado['teacher'].unique():
    turma_prof = df_result_filtrado[df_result_filtrado['teacher'] == i]
    horarios = turma_prof['horario'].unique()
    
    for h in horarios:
        filtro = professores_raw.loc[(professores_raw['TEACHER'] == i), h]
        
        if not filtro.empty:  # Verifica se existe algum valor antes de acessar
            valor = filtro.values[0]
            if valor == 0:
                print(f"Professor {i} não pode dar aulas no horário {h}")
        else:
            print(f"Atenção: Professor {i} não tem informações registradas para o horário {h}")

In [11]:
#professor nao pode dar aula no estagio
for i in df_result_filtrado['teacher'].unique():
    turma_prof = df_result_filtrado[df_result_filtrado['teacher'] == i]
    stages = turma_prof['stage'].unique()
    
    for h in stages:
        if h not in professores_raw.columns:  # Verifica se 'h' é uma coluna válida
            print(f"Atenção: '{h}' não é uma coluna em professores_raw. Verifique os dados.")
            continue
        
        filtro = professores_raw.loc[professores_raw['TEACHER'] == i, h]

        if not filtro.empty:  # Verifica se existe algum valor antes de acessar
            valor = filtro.values[0]
            if valor == 0:
                print(f"Professor {i} não pode dar aulas no estágio {h}")
        else:
            print(f"Atenção: Professor {i} não tem informações registradas para o estágio {h}")



Atenção: 'CONV' não é uma coluna em professores_raw. Verifique os dados.
Atenção: 'CONV' não é uma coluna em professores_raw. Verifique os dados.
Professor DIEGO - ESP não pode dar aulas no estágio ESTAGIO_1
Atenção: 'CONV' não é uma coluna em professores_raw. Verifique os dados.
Atenção: 'CONV' não é uma coluna em professores_raw. Verifique os dados.
Atenção: 'CONV' não é uma coluna em professores_raw. Verifique os dados.


In [14]:
#professor nao pode dar aula na modalidade
for i in df_result_filtrado['teacher'].unique():
    turma_prof = df_result_filtrado[df_result_filtrado['teacher'] == i]
    modality_list = [ 'Espanhol','Kids','CONV - Ing Prep','CONV - Ing Intemed',
                          'CONV - Ing Avançado','CONV - Esp Prep','CONV - Esp Intemed',
                          'CONV - Esp Avançado','MBA']
    for m in modality_list:
        modalidade_prof = turma_prof[turma_prof['modalidade']==m]
        modalidede_pod = professores_raw.loc[professores_raw['TEACHER']==i][m].values[0]
        if modalidade_prof.shape[0]>0:
            if modalidede_pod == 0:
                print(f"Professor {i} não pode dar aulas na modalidade {m}")

In [ ]:
modality_list = [ 'Espanhol','Kids','CONV - Ing Prep','CONV - Ing Intemed',
                          'CONV - Ing Avançado','CONV - Esp Prep','CONV - Esp Intemed',
                          'CONV - Esp Avançado','MBA']

In [12]:
for i in df_result_filtrado['teacher'].unique():
    turma_prof = df_result_filtrado[df_result_filtrado['teacher'] == i]
    modalidade = turma_prof['modalidade'].unique()
    print(modalidade)
    
    for h in horarios:
        filtro = professores_raw.loc[(professores_raw['TEACHER'] == i), h]
        
        if not filtro.empty:  # Verifica se existe algum valor antes de acessar
            valor = filtro.values[0]
            if valor == 0:
                print(f"Professor {i} não pode dar aulas no horário {h}")
        else:
            print(f"Atenção: Professor {i} não tem informações registradas para o horário {h}")

['Inglês']
Professor ALINE não pode dar aulas no horário 20:00:00
['MBA']
['Inglês']
['Inglês' 'Kids']
['Inglês']
['Inglês' 'CONV - Ing Intermed' 'CONV - Ing Avançado']
['Inglês']
['Inglês' 'Kids' 'CONV - Ing Avançado']
Professor CENKAY não pode dar aulas no horário 19:00:00
['Inglês']
['Kids' 'Inglês']
['Espanhol']
['Inglês']
['MBA']
['CONV - Ing Intermed']
['Espanhol']
['Inglês']
['Inglês']
['Inglês']
['Inglês']
['Inglês' 'CONV - Ing Intermed' 'Espanhol']
['Inglês' 'CONV - Ing Intermed']
['Espanhol']
['Inglês']
['Inglês']


In [13]:
df_final,aulas_nao_alocadas = transform_alocation_dataframe(aulas_raw,base_alocada)
aulas_nao_alocadas

,nome grupo,horario,unidade,dias da semana,stage,livro,modalidade,grupo,n aulas,parag atual grupo,...,rescisao,permuta,bolsista,n_total_alunos,stage_formula,parag_final_formula,num_1_sempre,ultimo_professor,penultimo_professor,restricoes_professor
0,ABU DHABI EAD - 4X,19:00:00,Vicentina,2ª ● 3ª ● 4ª ● 5ª,11,NOVO,Inglês,Grupo,4,1210,...,0,0,0,9,ERRO,TURMA,1,-,NaN,NaN
1,EDINBURGH P - 3X,19:00:00,Jardim,2ª ● 4ª ● 5ª,6,NOVO,Inglês,Grupo,3,T6,...,0,1,0,5,ERRO,TURMA,1,-,NaN,NaN
2,ANDRE VIP CONV EAD - 2X,07:30:00,Vicentina,4ª ● 6ª,CONV,NaN,CONV - Ing Avançado,VIP,2,CONV (NO SHOW),...,0,0,0,1,ERRO,TURMA,1,-,NaN,NaN
3,AUCKLAND P - 4X,19:00:00,Satélite,2ª ● 3ª ● 4ª ● 5ª,4,NOVO,Inglês,Grupo,4,376,...,0,0,0,8,ERRO,TURMA,1,-,NaN,NaN
4,AUSTIN EAD - 3X,08:00:00,Satélite,Saturday - Triple,4,NOVO,Inglês,Grupo,3,367,...,1,0,0,4,ERRO,TURMA,1,-,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,WANAKA P - 4X,19:00:00,Jardim,2ª ● 3ª ● 4ª ● 5ª,10,NOVO,Inglês,Grupo,4,1020,...,1,0,0,5,ERRO,TURMA,1,-,NaN,NaN
145,WARRINGTON P - 4X,18:00:00,Satélite,2ª ● 3ª ● 4ª ● 5ª,3,NOVO,Inglês,Grupo,4,233,...,0,0,0,3,ERRO,TURMA,1,-,NaN,NaN
146,WILLIAMSBURGH P - 3X,19:00:00,Satélite,2ª ● 3ª ● 5ª,7,NOVO,Inglês,Grupo,3,670,...,1,0,0,9,ERRO,TURMA,1,-,NaN,NaN
147,WISCONSIN EAD - 3X,20:00:00,Jardim,2ª ● 4ª ● 5ª,2,NOVO,Inglês,Grupo,3,R2-2,...,0,0,0,7,ERRO,TURMA,1,-,NaN,NaN
